<a href="https://colab.research.google.com/github/Zardian18/GPT_Quotes/blob/master/GPT_quotes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [2]:
from google.colab import drive
drive.mount("/content/drive")

! mkdir ~/.kaggle
!cp /content/drive/MyDrive/kaggle/kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

Mounted at /content/drive


In [3]:
! kaggle datasets download -d abireltaief/english-quotes

  0% 0.00/667k [00:00<?, ?B/s]
100% 667k/667k [00:00<00:00, 136MB/s]


In [4]:
!unzip english-quotes.zip

Archive:  english-quotes.zip
  inflating: quotes.csv              
  inflating: quotes.json             
  inflating: quotes.jsonl            


In [25]:
%load_ext autoreload
%autoreload 2
import numpy as np
import json
import re
import string
from IPython.display import display, HTML

import tensorflow as tf
from tensorflow.keras import layers, models, losses, callbacks
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [72]:
VOCAB_SIZE = 8000
MAX_LEN = 50
EMBEDDING_DIM = 256
KEY_DIM = 512
N_HEADS = 4
FEED_FORWARD_DIM = 512
VALIDATION_SPLIT = 0.2
SEED = 42
LOAD_MODEL = False
BATCH_SIZE = 32
EPOCHS = 15

In [26]:
df = pd.read_csv("/content/quotes.csv")
df.head(5)

,Unnamed: 0,quote,author,tags
0,0,“Be yourself; everyone else is already taken.”,Oscar Wilde,"['be-yourself', 'gilbert-perreira', 'honesty',..."
1,1,"“I'm selfish, impatient and a little insecure....",Marilyn Monroe,"['best', 'life', 'love', 'mistakes', 'out-of-c..."
2,2,“Two things are infinite: the universe and hum...,Albert Einstein,"['human-nature', 'humor', 'infinity', 'philoso..."
3,3,"“So many books, so little time.”",Frank Zappa,"['books', 'humor']"
4,4,“A room without books is like a body without a...,Marcus Tullius Cicero,"['books', 'simile', 'soul']"


In [27]:
quotes=df["quote"]
quotes.head(5)

0       “Be yourself; everyone else is already taken.”
1    “I'm selfish, impatient and a little insecure....
2    “Two things are infinite: the universe and hum...
3                     “So many books, so little time.”
4    “A room without books is like a body without a...
Name: quote, dtype: object

In [30]:
quotes_text =[]
for i in range(len(quotes)):
  quotes_text.append(quotes[i])

In [32]:
quotes_text[:3]

['“Be yourself; everyone else is already taken.”',
 "“I'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can't handle me at my worst, then you sure as hell don't deserve me at my best.”",
 "“Two things are infinite: the universe and human stupidity; and I'm not sure about the universe.”"]

In [33]:
def pad_punctuation(s):
    s = re.sub(f"([{string.punctuation}, '\n'])", r" \1 ", s)
    s = re.sub(" +", " ", s)
    return s


text_data = [pad_punctuation(x) for x in quotes_text]

In [34]:
text_data[:3]

['“Be yourself ; everyone else is already taken . ”',
 "“I ' m selfish , impatient and a little insecure . I make mistakes , I am out of control and at times hard to handle . But if you can ' t handle me at my worst , then you sure as hell don ' t deserve me at my best . ”",
 "“Two things are infinite : the universe and human stupidity ; and I ' m not sure about the universe . ”"]

In [35]:
text_ds = (
    tf.data.Dataset.from_tensor_slices(text_data)
    .batch(BATCH_SIZE)
    .shuffle(1000)
)

In [36]:
text_ds

<_ShuffleDataset element_spec=TensorSpec(shape=(None,), dtype=tf.string, name=None)>

In [37]:
sum =0
for i in range(len(text_data)):
  words = text_data[i].split()
  sum += len(words)

print(f"Average words per quote: {sum/len(text_data)}")

Average words per quote: 38.77392344497608


In [45]:
vectorize_layer = layers.TextVectorization(
    standardize="lower",
    max_tokens=VOCAB_SIZE,
    output_mode="int",
    output_sequence_length=MAX_LEN + 1,
)

In [46]:
vectorize_layer.adapt(text_ds)
vocab = vectorize_layer.get_vocabulary()

In [47]:
for i, word in enumerate(vocab[:10]):
    print(f"{i}: {word}")

0: 
1: [UNK]
2: .
3: ,
4: the
5: you
6: ”
7: to
8: and
9: '


In [48]:
len(vocab)

7915

In [50]:
example_tokenised = vectorize_layer(text_data[1])
print(example_tokenised.numpy())

[  51    9  106 2054    3 3106    8   10  149 2208    2   11   87  471
    3   11   98   81   12  646    8   52  263  282    7 2231    2   26
   41    5   27    9   22 2231   37   52   34  619    3   94    5  327
   39  345   61    9   22  645   37   52   34]


In [51]:
def prepare_inputs(text):
    text = tf.expand_dims(text, -1)
    tokenized_sentences = vectorize_layer(text)
    x = tokenized_sentences[:, :-1]
    y = tokenized_sentences[:, 1:]
    return x, y


train_ds = text_ds.map(prepare_inputs)

In [52]:
def prepare_inputs(text):
    text = tf.expand_dims(text, -1)
    tokenized_sentences = vectorize_layer(text)
    x = tokenized_sentences[:, :-1]
    y = tokenized_sentences[:, 1:]
    return x, y


train_ds = text_ds.map(prepare_inputs)

In [54]:
example_input_output = train_ds.take(1).get_single_element()

In [58]:
example_input_output[0][0] # input

<tf.Tensor: shape=(50,), dtype=int64, numpy=
array([ 174,   20,   57,  830,   46, 6593, 6501,   66,  121, 2250,  260,
         56,   42,   33,    4,  233,  153,    2,    6,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0])>

In [57]:
example_input_output[1][0] #output

<tf.Tensor: shape=(50,), dtype=int64, numpy=
array([  20,   57,  830,   46, 6593, 6501,   66,  121, 2250,  260,   56,
         42,   33,    4,  233,  153,    2,    6,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0])>

In [59]:
example_input_output

(<tf.Tensor: shape=(32, 50), dtype=int64, numpy=
 array([[ 174,   20,   57, ...,    0,    0,    0],
        [ 284,    9,   22, ...,    0,    0,    0],
        [ 267,    5,   44, ...,    0,    0,    0],
        ...,
        [ 913, 1020,   75, ...,    0,    0,    0],
        [ 129,   13,   10, ...,    0,    0,    0],
        [ 129,    9,   23, ...,    2,    6,    0]])>,
 <tf.Tensor: shape=(32, 50), dtype=int64, numpy=
 array([[  20,   57,  830, ...,    0,    0,    0],
        [   9,   22,  227, ...,    0,    0,    0],
        [   5,   44,    3, ...,    0,    0,    0],
        ...,
        [1020,   75,    4, ...,    0,    0,    0],
        [  13,   10,  178, ...,    0,    0,    0],
        [   9,   23,   19, ...,    6,    0,    0]])>)

In [60]:
def causal_attention_mask(batch_size, n_dest, n_src, dtype):
    i = tf.range(n_dest)[:, None]
    j = tf.range(n_src)
    m = i >= j - n_src + n_dest
    mask = tf.cast(m, dtype)
    mask = tf.reshape(mask, [1, n_dest, n_src])
    mult = tf.concat(
        [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], 0
    )
    return tf.tile(mask, mult)

In [61]:
np.transpose(causal_attention_mask(1, 10, 10, dtype=tf.int32)[0])

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]], dtype=int32)

In [62]:
class TransformerBlock(layers.Layer):
    def __init__(self, num_heads, key_dim, embed_dim, ff_dim, dropout_rate=0.1):
        super(TransformerBlock, self).__init__()
        self.num_heads = num_heads
        self.key_dim = key_dim
        self.embed_dim = embed_dim
        self.ff_dim = ff_dim
        self.dropout_rate = dropout_rate
        self.attn = layers.MultiHeadAttention(
            num_heads, key_dim, output_shape=embed_dim
        )
        self.dropout_1 = layers.Dropout(self.dropout_rate)
        self.ln_1 = layers.LayerNormalization(epsilon=1e-6)
        self.ffn_1 = layers.Dense(self.ff_dim, activation="relu")
        self.ffn_2 = layers.Dense(self.embed_dim)
        self.dropout_2 = layers.Dropout(self.dropout_rate)
        self.ln_2 = layers.LayerNormalization(epsilon=1e-6)

    def call(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size = input_shape[0]
        seq_len = input_shape[1]
        causal_mask = causal_attention_mask(
            batch_size, seq_len, seq_len, tf.bool
        )
        attention_output, attention_scores = self.attn(
            inputs,
            inputs,
            attention_mask=causal_mask,
            return_attention_scores=True,
        )
        attention_output = self.dropout_1(attention_output)
        out1 = self.ln_1(inputs + attention_output)
        ffn_1 = self.ffn_1(out1)
        ffn_2 = self.ffn_2(ffn_1)
        ffn_output = self.dropout_2(ffn_2)
        return (self.ln_2(out1 + ffn_output), attention_scores)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "key_dim": self.key_dim,
                "embed_dim": self.embed_dim,
                "num_heads": self.num_heads,
                "ff_dim": self.ff_dim,
                "dropout_rate": self.dropout_rate,
            }
        )
        return config

In [63]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, max_len, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.max_len = max_len
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.token_emb = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.pos_emb = layers.Embedding(input_dim=max_len, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "max_len": self.max_len,
                "vocab_size": self.vocab_size,
                "embed_dim": self.embed_dim,
            }
        )
        return config

In [73]:
inputs = layers.Input(shape=(None,), dtype=tf.int32)
x = TokenAndPositionEmbedding(MAX_LEN, VOCAB_SIZE, EMBEDDING_DIM)(inputs)
x, attention_scores = TransformerBlock(N_HEADS, KEY_DIM, EMBEDDING_DIM, FEED_FORWARD_DIM)(x)
outputs = layers.Dense(VOCAB_SIZE, activation="softmax")(x)

gpt = models.Model(inputs= inputs, outputs= [outputs, attention_scores])
gpt.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 token_and_position_embeddi  (None, None, 256)         2060800   
 ng_1 (TokenAndPositionEmbe                                      
 dding)                                                          
                                                                 
 transformer_block_1 (Trans  ((None, None, 256),       2367488   
 formerBlock)                 (None, 4, None, None))             
                                                                 
 dense_5 (Dense)             (None, None, 8000)        2056000   
                                                                 
Total params: 6484288 (24.74 MB)
Trainable params: 6484288 (24.74 MB)
Non-trainable params: 0 (0.00 Byte)
___________________

In [74]:
gpt.compile("adam", loss=[losses.SparseCategoricalCrossentropy(), None])

In [79]:
import random
class TextGenerator(callbacks.Callback):
    def __init__(self, index_to_word, top_k=10):
        self.index_to_word = index_to_word
        self.word_to_index = {
            word: index for index, word in enumerate(index_to_word)
        }

    def sample_from(self, probs, temperature):
        probs = probs ** (1 / temperature)
        probs = probs / np.sum(probs)
        return np.random.choice(len(probs), p=probs), probs

    def generate(self, start_prompt, max_tokens, temperature):
        start_tokens = [
            self.word_to_index.get(x, 1) for x in start_prompt.split()
        ]
        sample_token = None
        info = []
        while len(start_tokens) < max_tokens and sample_token != 0:
            x = np.array([start_tokens])
            y, att = self.model.predict(x, verbose=0)
            sample_token, probs = self.sample_from(y[0][-1], temperature)
            info.append(
                {
                    "prompt": start_prompt,
                    "word_probs": probs,
                    "atts": att[0, :, -1, :],
                }
            )
            start_tokens.append(sample_token)
            start_prompt = start_prompt + " " + self.index_to_word[sample_token]
        print(f"\ngenerated text:\n{start_prompt}\n")
        return info

    def on_epoch_end(self, epoch, logs=None):
      wordx = random.choice(vocab)
      self.generate(wordx, max_tokens=50, temperature=1.0)

In [80]:
model_checkpoint_callback = callbacks.ModelCheckpoint(
    filepath="./checkpoint/checkpoint.ckpt",
    save_weights_only=True,
    save_freq="epoch",
    verbose=0,
)

tensorboard_callback = callbacks.TensorBoard(log_dir="./logs")

# Tokenize starting prompt
text_generator = TextGenerator(vocab)

In [81]:
gpt.fit(
    train_ds,
    epochs=50,
    callbacks=[model_checkpoint_callback, tensorboard_callback, text_generator],
)

Epoch 1/50
 5/79 [>.............................] - ETA: 2s - loss: 2.3599 - dense_5_loss: 2.3599

79/79 [==============================] - ETA: 0s - loss: 2.2654 - dense_5_loss: 2.2654
generated text:
individual - gemstones ! how much thing , some part of god should be like your dad to never sleep . you are up all . if you . ” 

79/79 [==============================] - 5s 61ms/step - loss: 2.2654 - dense_5_loss: 2.2654
Epoch 2/50
77/79 [============================>.] - ETA: 0s - loss: 2.0268 - dense_5_loss: 2.0268
generated text:
neither is intense person who 

79/79 [==============================] - 3s 33ms/step - loss: 2.0281 - dense_5_loss: 2.0281
Epoch 3/50
79/79 [==============================] - ETA: 0s - loss: 1.8212 - dense_5_loss: 1.8212
generated text:
tremendous to be prisoner all of all that life : where i want to got to get to have my own - be worth fond of terrorjust ” 

79/79 [==============================] - 4s 49ms/step - loss: 1.8212 - dense_5_loss: 1.8212
Epoch 4/50
78/79 [============================>.] - ETA: 0s - loss: 1.6158 - dense_5_loss: 1.6158
generate

In [78]:
text_data[10]

"“In three words I can sum up everything I ' ve learned about life : it goes on . ”"

In [82]:
def print_probs(info, vocab, top_k=5):
    for i in info:
        highlighted_text = []
        for word, att_score in zip(
            i["prompt"].split(), np.mean(i["atts"], axis=0)
        ):
            highlighted_text.append(
                ''
                + word
                + ""
            )
        highlighted_text = " ".join(highlighted_text)
        display(HTML(highlighted_text))

        word_probs = i["word_probs"]
        p_sorted = np.sort(word_probs)[::-1][:top_k]
        i_sorted = np.argsort(word_probs)[::-1][:top_k]
        for p, i in zip(p_sorted, i_sorted):
            print(f"{vocab[i]}:   \t{np.round(100*p,2)}%")
        print("--------\n")

In [91]:
info = text_generator.generate(
    "there", max_tokens=50, temperature=1.0
)


generated text:
there are troubles to than we can imagine . because they are the reason . ” 



In [92]:
print_probs(info, vocab)

is:   	41.24%
are:   	32.63%
':   	20.61%
may:   	1.27%
you:   	0.82%
--------



no:   	49.14%
a:   	6.26%
always:   	6.2%
two:   	3.27%
wounds:   	2.91%
--------



of:   	69.85%
for:   	9.68%
to:   	8.3%
when:   	4.64%
and:   	1.15%
--------



more:   	37.08%
sex:   	22.0%
come:   	10.82%
than:   	10.01%
me:   	7.01%
--------



we:   	26.92%
and:   	23.47%
they:   	18.12%
a:   	10.96%
more:   	10.29%
--------



...

no:   	48.06%
the:   	27.07%
one:   	12.11%
.:   	10.87%
only:   	0.66%
--------



one:   	78.08%
perfect:   	8.79%
pain:   	4.32%
reason:   	3.67%
small:   	0.74%
--------



.:   	96.63%
and:   	1.67%
is:   	1.24%
for:   	0.19%
of:   	0.13%
--------



”:   	100.0%
they:   	0.0%
genius:   	0.0%
maybe:   	0.0%
just:   	0.0%
--------



:   	100.0%
words:   	0.0%
t:   	0.0%
you:   	0.0%
':   	0.0%
--------



In [100]:
info = text_generator.generate(
    "believe", max_tokens=50, temperature=1.0
)


generated text:
believe in god , or joy is the kind of creating strength , but yourself . ” 



In [103]:
info = text_generator.generate(
    "believe", max_tokens=50, temperature=0.5
)


generated text:
believe in everything happens when people change that life is your way while you think it . ” 



In [141]:
info = text_generator.generate(
    "success", max_tokens=50, temperature=1
)


generated text:
success is not only the friend with hope a in the world acknowledges your strength . ” 



In [147]:
if "power" in vocab:
  print("Yes")

else:
  print("No")

Yes


In [144]:
info = text_generator.generate(
    "wealth, fame, power", max_tokens=50, temperature=1
)


generated text:
wealth, fame, power are all these things : right words , and human life is . ” 



In [155]:
info = text_generator.generate(
    "you are", max_tokens=50, temperature=1
)


generated text:
you are , and always have been , my dream . ” 



In [157]:
info = text_generator.generate(
    "first things", max_tokens=50, temperature=1
)


generated text:
first things happen to you and so bad , you ' re broken open . ” 



In [176]:
info = text_generator.generate(
    "it 's stupid", max_tokens=50, temperature=1
)


generated text:
it 's stupid , so we can ' t get part of madness . ” 



In [180]:
info = text_generator.generate(
    "don' t laugh", max_tokens=50, temperature=1
)


generated text:
don' t laugh you lose your footing . ” 



In [181]:
info = text_generator.generate(
    "gym", max_tokens=50, temperature=1
)


generated text:
gym love is not madness . ” 



In [183]:
info = text_generator.generate(
    "gym", max_tokens=50, temperature=1
)


generated text:
gym must be a mermaid , rango . i have no fear of depths and a great fear of shallow living . ” 

